# Stochastic Hill Climbers

We'll code the workshop teaser!
The idea is simple:

- Start with 50 8-vertex polygons randomly in space
  - Assign a color and transparency to each polygon
- Randomly nudge one of the vertices
  - If resulting image is closer to the target image, accept change, otherwise reject
- Iterate until convergence

In [ ]:
obama = ImageResize[Import["https://www.beyonddream.com/images/product/23892024.jpg"], 500]

Find the 50 'dominant' colors:

In [ ]:
QuaCol[i_, n_] := RGBColor /@ Union[Flatten[ImageData[ColorQuantize[i, n]], 1]]
colors = Sort[QuaCol[obama, 50]]

Make a random polygon:

In [ ]:
pixelPts = Tuples[Range[1, #, 3] & /@ ImageDimensions[obama]];

polygon[integers_, col_] := 
 Block[{order = First@FindCurvePath[pixelPts[[integers]]]},
  {Opacity[0.5], col, Polygon[integers[[order]]]}]
  

polygon[RandomChoice[Range[Length[pixelPts]], 8], Red]
Graphics[GraphicsComplex[pixelPts, %]]

Make a visualization function:

In [ ]:
rasterize[dna_, cols_ : colors] := Rasterize[
  Graphics[GraphicsComplex[pixelPts, 
    MapThread[polygon, {Partition[dna, 8], cols}]], ImageSize -> 500, 
   PlotRange -> {{1, 500}, {1, 417}}]]

Make a collection of random polygons:

In [ ]:
rasterize[RandomSample[Range[Length[pixelPts]], 50 8]]

Compute an image distance metric:

In [ ]:
imageDistance[string_] := 
 ImageDistance[obama, rasterize[string], 
  DistanceFunction -> SquaredEuclideanDistance]
  
imageDistance[RandomSample[Range[Length[pixelPts]], 50 8]]

'Mutate' by randomly 'nudging' a corner:

In [ ]:
mutate[m_][dna_] := 
 Block[{neighs = RandomInteger[{1, Length[dna]}, m]},
  ReplacePart[dna, 
   Thread[neighs -> RandomSample[Range[Length[pixelPts]], m]]]]

Accept/reject 'mutation' and 'evolve':

In [ ]:
iterate[m_] := Block[{mutation, metric, result},
  mutation = mutate[m][dna];
  metric = imageDistance[mutation];
  result = If[metric < previous,
    previous = metric;
    mutation, dna];
  n++;
  result]

**Note:** see the Wolfram cloud notebook for an interactive version of the minimization!

In [ ]:
n = 0;
dna = RandomSample[Range[Length[pixelPts]], 50 8];
og = previous = imageDistance[dna]

In [ ]:
Show[rasterize[dna], 
 PlotLabel -> Style[StringTemplate["Scaled Image Distance: `1`"][previous/og], 24,Black]]

In [ ]:
Do[dna = iterate[1], {1000000}]

![teaser](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/PR/teaser.gif)